# Spark _DataFrame_

## Oberview

The previous section showed you how to read a CSV file into Spark. The result is stored in a data structured called
DataFrame. A Spark DataFrame is a like a distributed in-memory table [1]. As a table like structure it has
columns and each column has a specific data type.

DataFrames play a key role in developing Spark applications. In this section we will go over the core elements you need
to know in order to work efficiently with them.

## Spark _DataFrame_

A Spark _DataFrame_ is a like a distributed in-memory table [1]. As a table like structure it has
columns and each column has a specific data type. DataFrames are immutable and this allows Spark to keep a lineage of all the
transformations applied on them. A DataFrame has a certain schema [1]. A schema defines the column names and the associated data types.
When reading data from a specific source we can either let Spark infer the schema, just like we did in the previous section, or explicitly
specifying the schema. The latter approach has two distinct benefits [1]:

- Inferring data can be tricky and hence time consuming; Spark needs to creat a separate job, read a large portion of the data and then infer the schema
- Providing the schema means we can infer quickly if the data doesn't match the proposed schema.

Let's see how an application can provide the schema of a dataset in Spark. There are two wasy to do so:

- Employ a data definition language (DDL) string
- Define it programmatically

The script below shows the first approach

```python
"""Loads a CSV file into Spark
"""

"""Convert a csv file to parquet format.
This application is meant to be submitted on
Spark for execution

"""
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pathlib import Path
import sys
import numpy as np

APP_NAME = "LOAD_CSV_FILE_TO_SPARK"

if __name__ == '__main__':

    if len(sys.argv) != 2:
        print("Usage: filename <file>", file=sys.stderr)

    # get a spark session
    spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

    # read the filename from the commandline

    # where is the file to read
    filename = Path(sys.argv[1])

    print(f"Loading file {filename}")

    # set the schema using DDL
    schema = "`OBJECTID` INT, `TRIPTYPE` INT, "\
             "`PROVIDERNAME` STRING, `FAREAMOUNT` FLOAT,"\
             "`GRATUITYAMOUNT` FLOAT, `SURCHARGEAMOUNT` FLOAT, "\
             "`EXTRAFAREAMOUNT` FLOAT, `TOLLAMOUNT` FLOAT, "\
             "`TOTALAMOUN` FLOAT, `PAYMENTTYPE` STRING,"\
             "`ORIGINCITY` STRING, `ORIGINSTATE` STRING,"\
             "`ORIGINZIP` STRING, `DESTINATIONCITY` STRING,"\
             "`DESTINATIONSTATE` STRING, `DESTINATIONZIP` STRING,"\
             "`MILEAGE` STRING, `DURATION` STRING,"\
             "`ORIGIN_BLOCK_LATITUDE` STRING, `ORIGIN_BLOCK_LONGITUDE` STRING,"\
             "`ORIGIN_BLOCKNAME` STRING, `DESTINATION_BLOCK_LATITUDE` STRING,"\
             "`DESTINATION_BLOCK_LONGITUDE` STRING, `DESTINATION_BLOCKNAME` STRING,"\
             "`AIRPORT` STRING, `ORIGINDATETIME_TR` STRING, `DESTINATIONDATETIME_TR` STRING"

    # read the file into a Spark DataFrame
    # the schema is inferred and it assumes that
    # a header is contained
    csv_df = (spark.read.format("csv")
              .option("header", "true")
              .option("inferSchema", False)
              .option("delimiter", "|")
              .schema(schema)
              .load(str(filename)))


    print(f"Schema used {csv_df.printSchema()}")
    spark.stop()

```

The output of the script will show

```
|-- OBJECTID: integer (nullable = true)
 |-- TRIPTYPE: integer (nullable = true)
 |-- PROVIDERNAME: string (nullable = true)
 |-- FAREAMOUNT: float (nullable = true)
 |-- GRATUITYAMOUNT: float (nullable = true)
 |-- SURCHARGEAMOUNT: float (nullable = true)
 |-- EXTRAFAREAMOUNT: float (nullable = true)
 |-- TOLLAMOUNT: float (nullable = true)
 |-- TOTALAMOUN: float (nullable = true)
 |-- PAYMENTTYPE: string (nullable = true)
 |-- ORIGINCITY: string (nullable = true)
 |-- ORIGINSTATE: string (nullable = true)
 |-- ORIGINZIP: string (nullable = true)
 |-- DESTINATIONCITY: string (nullable = true)
 |-- DESTINATIONSTATE: string (nullable = true)
 |-- DESTINATIONZIP: string (nullable = true)
 |-- MILEAGE: string (nullable = true)
 |-- DURATION: string (nullable = true)
 |-- ORIGIN_BLOCK_LATITUDE: string (nullable = true)
 |-- ORIGIN_BLOCK_LONGITUDE: string (nullable = true)
 |-- ORIGIN_BLOCKNAME: string (nullable = true)
 |-- DESTINATION_BLOCK_LATITUDE: string (nullable = true)
 |-- DESTINATION_BLOCK_LONGITUDE: string (nullable = true)
 |-- DESTINATION_BLOCKNAME: string (nullable = true)
 |-- AIRPORT: string (nullable = true)
 |-- ORIGINDATETIME_TR: string (nullable = true)
 |-- DESTINATIONDATETIME_TR: string (nullable = true)
```

Programmatically, specifying the schema is more involved as one needs to use the data types from Spark. A snapshot 
how to do this is shown below.


```python
...

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType # programatically specify the schema
...

    # specify the schema programmatically
    schema = StructType([StructField("OBJECTID", IntegerType(), False),
                         StructField("PROVIDERNAME", StringType(), False),
                         
                         ...
                         ])
```

### Create a DataFrame

So far we have been using the read methods available in Spark in order to create a DataFrame. However, we can explicitly create a 
DataFrame without reading a file. This is shown in the script below.

```python
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType #programatically specify the schema


APP_NAME = "LOAD_CSV_FILE_TO_SPARK"

if __name__ == '__main__':

    # get a spark session
    spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

    # set the schema using DDL
    schema = "`OBJECTID` INT, `TRIPTYPE` INT, `PROVIDERNAME` STRING, `FAREAMOUNT` FLOAT"

    # specify the schema programmatically
    schema = StructType([StructField("OBJECTID", IntegerType(), False),
                         StructField("TRIPTYPE", IntegerType(), False),
                         StructField("PROVIDERNAME", StringType(), False),
                         StructField("FAREAMOUNT", FloatType(), False)])

    data = [[1, 3, "NEW-YORK", 20.0],
            [2, 2, "CAMBRIDGE", 18.2],
            [3, 3, "NEW-YORK", 20.0],
            [4, 2, "LONDON", 25.0],
            [5, 2, "OXFORD", 15.0]]


    df = spark.createDataFrame(data, schema)
    df.show()

    print(f"Schema used {df.printSchema()}")

    spark.stop()

```

Running the script produces the following

```
|OBJECTID|TRIPTYPE|PROVIDERNAME|FAREAMOUNT|
+--------+--------+------------+----------+
|       1|       3|    NEW-YORK|      20.0|
|       2|       2|   CAMBRIDGE|      18.2|
|       3|       3|    NEW-YORK|      20.0|
|       4|       2|      LONDON|      25.0|
|       5|       2|      OXFORD|      15.0|
+--------+--------+------------+----------+

root
 |-- OBJECTID: integer (nullable = false)
 |-- TRIPTYPE: integer (nullable = false)
 |-- PROVIDERNAME: string (nullable = false)
 |-- FAREAMOUNT: float (nullable = false)

```

A DataFrame consists of Row objects. A Row is a generic object containing one or more columns [1]. A Row can be instantiated and used
to create a DataFrame. This is shown in the script below



```
from pyspark.sql import SparkSession
from pyspark.sql import Row
import sys


APP_NAME = "LOAD_CSV_FILE_TO_SPARK"

if __name__ == '__main__':

    if len(sys.argv) != 2:
        print("Usage: filename <file>", file=sys.stderr)

    # get a spark session
    spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

    # read the filename from the commandline
    rows = [Row("Alex", "Western Road", "Germany", 2),
            Row("Stewart", "New Road", "France", 64),
            Row("Elina", "Hemmel Area", "Spain", 146)]

    # create the DataFrame
    df = spark.createDataFrame(rows, ["Name", "Street Name", "Country", "Number"])
    df.show()

    spark.stop()
```

The output of the script is as follows

```
+-------+------------+-------+------+
|   Name| Street Name|Country|Number|
+-------+------------+-------+------+
|   Alex|Western Road|Germany|     2|
|Stewart|    New Road| France|    64|
|  Elina| Hemmel Area|  Spain|   146|
+-------+------------+-------+------+

```

### Columns

We saw above how to create a DataFrame using the ```createDataFrame``` function or loading from a data file.
A DataFrame consists of Row and Column objects. One can perform operations on a Column object by using the ```expr()``` function.
Some examples are shown below.

```python
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import expr, col
import sys


APP_NAME = "LOAD_CSV_FILE_TO_SPARK"

if __name__ == '__main__':

    if len(sys.argv) != 2:
        print("Usage: filename <file>", file=sys.stderr)

    # get a spark session
    spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

    # read the filename from the commandline
    rows = [Row(180.0, 85.0, 35, "M"),
            Row(175.5, 75.5, 25, "M"),
            Row(165.3, 55.3, 19, "F")]

    # create the DataFrame
    df = spark.createDataFrame(rows, ["Height", "Weight", "Age", "Sex"])
    df.show()

    # use expr function
    df.select(expr("Height * 5")).show()

    #...or use column
    df.select("Height", col("Height") * 5, "Weight", col("Weight") * 2, "Age", "Sex").show()

    spark.stop()

```
Running the script produces the following

```
+------+------+---+---+
|Height|Weight|Age|Sex|
+------+------+---+---+
| 180.0|  85.0| 35|  M|
| 175.5|  75.5| 25|  M|
| 165.3|  55.3| 19|  F|
+------+------+---+---+

+------------+
|(Height * 5)|
+------------+
|       900.0|
|       877.5|
|       826.5|
+------------+

+------+------------+------+------------+---+---+
|Height|(Height * 5)|Weight|(Weight * 2)|Age|Sex|
+------+------------+------+------------+---+---+
| 180.0|       900.0|  85.0|       170.0| 35|  M|
| 175.5|       877.5|  75.5|       151.0| 25|  M|
| 165.3|       826.5|  55.3|       110.6| 19|  F|
+------+------------+------+------------+---+---+

```

#### Dropping columns

Often we may not want to use certain columns. We can drop these from the workind DataFrame easily

```python

drop_columns = ["relative_location", "friend"]
update_dataframe.drop(*drop_columns)

```

## DataFrame readers and writers

In order to perform data operations on a _DataFrame_, we first need to populate it with data. Spark supports reading of
data from various siurces such as 

- CSV,
- Json,
- Parquet,
- Avro,
- Text

It also supports loading data from various DB managers, Apache Kafka and Kinesis. This is done via the _DataFrameReader_ class. We have already used this class when
we did

```
spark.read.csv(options)
```

In a similar fashion we can use the _DataFrameWriter_ class. We will see more examples of these two classes as we discuss more functionality in Spark.

## Summary

In this section we gave an introduction the Spark's DataFrame, Row and columns. We saw how to create a DataFrame using the
```createDataFrame```. Also how to use ```Row``` in order to create a DataFrame. A DataFrame is a collection consisting of Row objects.
Spark allows us to perform various operations on columns using for example the ```expr()``` function. We will see more examples of applying operations
on columns in later sections.

## References

1. Jules S. Damji, Brooke Wenig, Tathagata Das, Deny Lee, _Learning Spark. Lighting-fasts data analytics_, 2nd Edition, O'Reilly.